In [1]:
#coding=utf-8


import pandas as pd
import xgboost as xgb
import sys,random
import _pickle as cPickle
import os

# os.mkdir('featurescore')
# os.mkdir('model')
# os.mkdir('preds')

#load data
train_x = pd.read_csv("C:/Users/tianjiayang/loan_predicting/data/train_x_rank.csv")
train_y = pd.read_csv("C:/Users/tianjiayang/loan_predicting/data/train_y.csv")
train_xy = pd.merge(train_x,train_y,on='uid')
y = train_xy.y
train_x= train_xy.drop(["uid",'y'],axis=1)
X = train_x/15000.0
dtrain = xgb.DMatrix(X, label=y)
    
test = pd.read_csv("C:/Users/tianjiayang/loan_predicting/data/test_x_rank.csv")
test_uid = test.uid
test = test.drop("uid",axis=1)
test_x = test/5000.0
dtest = xgb.DMatrix(test_x)
print('finish loading data')


def pipeline(iteration,random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight):
   #参数
    params={
    	'booster':'gbtree',#tree
    	'objective': 'rank:pairwise', #是否可以用pairwise？
        
#         “reg:linear” –线性回归。
# “reg:logistic” –逻辑回归。
# “binary:logistic” –二分类的逻辑回归问题，输出为概率。
# “binary:logitraw” –二分类的逻辑回归问题，输出的结果为wTx。
# “count:poisson” –计数问题的poisson回归，输出结果为poisson分布。
# 在poisson回归中，max_delta_step的缺省值为0.7。(used to safeguard optimization)
# “multi:softmax” –让XGBoost采用softmax目标函数处理多分类问题，同时需要设置参数num_class（类别个数）
# “multi:softprob” –和softmax一样，但是输出的是ndata * nclass的向量，可以将该向量reshape成ndata行nclass列的矩阵。没行数据表示样本所属于每个类别的概率。
# “rank:pairwise” –set XGBoost to do ranking task by minimizing the pairwise loss
    	'scale_pos_weight': float(len(y)-sum(y))/float(sum(y)),
        'eval_metric': 'auc',
        #minimum loss reduction required to make a further partition on a leaf node of the tree. 
        #the larger, the more conservative the algorithm will be.
    	'gamma':gamma,
    	'max_depth':max_depth,
        #L2的力度
    	'lambda':lambd,
        #用于训练模型的子样本占整个样本集合的比例。如果设置为0.5则意味着XGBoost将随机的冲整个样本集合中随机的抽取出50%的子样本建立树模型
        'subsample':subsample,
        #列采样
        'colsample_bytree':colsample_bytree,
        'min_child_weight':min_child_weight, 
        #更新过程中用到的收缩步长
        'eta': 0.04,
    	'seed':random_seed,
    	'nthread':8
        }
    
    watchlist  = [(dtrain,'train')]
    print('start training')
    model = xgb.train(params,dtrain,num_boost_round=1350,evals=watchlist)
    print('done training')
    model.save_model('./model/xgb{0}.model'.format(iteration))
    
    #predict test set
    test_y = model.predict(dtest)
    print('done predicting')
    test_result = pd.DataFrame(columns=["uid","score"])
    test_result.uid = test_uid
    test_result.score = test_y
    test_result.to_csv("./preds/xgb{0}.csv".format(iteration),index=None,encoding='utf-8')
    
    #save feature score
    #This is a metric that simply sums up how many times each feature is split on.
    feature_score = model.get_fscore()
    feature_score = sorted(feature_score.items(), key=lambda x:x[1],reverse=True)
    fs = []
    for (key,value) in feature_score:
        fs.append("{0},{1}\n".format(key,value))
    
    with open('./featurescore/feature_score_{0}.csv'.format(iteration),'w') as f:
        f.writelines("feature,score\n")
        f.writelines(fs)



if __name__ == "__main__":
    random_seed_list = [i for i in range(100,200,10)]
    gamma = [i/1000.0 for i in range(100,200,10)]
    max_depth = [6,7,8]
    lambd = [i for i in range(100,200,10)]
    subsample = [i/1000.0 for i in range(500,700,20)]
    colsample_bytree = [i/1000.0 for i in range(250,350,10)]
    min_child_weight = [i/1000.0 for i in range(200,300,10)]
    random.shuffle(random_seed_list)
    random.shuffle(gamma)
    random.shuffle(max_depth)
    random.shuffle(lambd)
    random.shuffle(subsample)
    random.shuffle(colsample_bytree)
    random.shuffle(min_child_weight)
    
    #save params for reproducing
    with open('params.pkl','wb') as f:
        cPickle.dump((random_seed_list,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight),f)
    
    #to reproduce my result, uncomment following lines
    """
    with open('params_for_reproducing.pkl','r') as f:
        random_seed,gamma,max_depth,lambd,subsample,colsample_bytree,min_child_weight = cPickle.load(f)    
    """

    #train 10 xgb
    for i in range(10):
        pipeline(i,random_seed_list[i],gamma[i],max_depth[i%3],lambd[i],subsample[i],colsample_bytree[i],min_child_weight[i])

finish loading data
start training
[22:25:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 2 pruned nodes, max_depth=7
[0]	train-auc:0.688198
[22:25:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 176 extra nodes, 0 pruned nodes, max_depth=7
[1]	train-auc:0.741044
[22:25:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 2 pruned nodes, max_depth=7
[2]	train-auc:0.756499
[22:25:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 180 extra nodes, 2 pruned nodes, max_depth=7
[3]	train-auc:0.768221
[22:25:32] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 2 pruned nodes, max_depth=7
[4]	train-auc:0.776134
[22:25:33] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruni

[22:25:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 4 pruned nodes, max_depth=7
[47]	train-auc:0.855722
[22:25:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 6 pruned nodes, max_depth=7
[48]	train-auc:0.855894
[22:25:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 194 extra nodes, 2 pruned nodes, max_depth=7
[49]	train-auc:0.857863
[22:25:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 6 pruned nodes, max_depth=7
[50]	train-auc:0.859015
[22:25:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 192 extra nodes, 6 pruned nodes, max_depth=7
[51]	train-auc:0.860623
[22:25:39] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 172 extra nod

[22:25:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 150 extra nodes, 6 pruned nodes, max_depth=7
[94]	train-auc:0.897634
[22:25:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 210 extra nodes, 8 pruned nodes, max_depth=7
[95]	train-auc:0.898875
[22:25:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 208 extra nodes, 6 pruned nodes, max_depth=7
[96]	train-auc:0.899966
[22:25:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 164 extra nodes, 6 pruned nodes, max_depth=7
[97]	train-auc:0.900539
[22:25:45] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 6 pruned nodes, max_depth=7
[98]	train-auc:0.901585
[22:25:46] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 200 extra nod

[22:25:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 12 pruned nodes, max_depth=7
[141]	train-auc:0.931553
[22:25:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 212 extra nodes, 8 pruned nodes, max_depth=7
[142]	train-auc:0.932343
[22:25:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 144 extra nodes, 4 pruned nodes, max_depth=7
[143]	train-auc:0.932727
[22:25:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 160 extra nodes, 10 pruned nodes, max_depth=7
[144]	train-auc:0.933393
[22:25:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 166 extra nodes, 2 pruned nodes, max_depth=7
[145]	train-auc:0.933955
[22:25:52] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 196 ex

[22:25:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 178 extra nodes, 8 pruned nodes, max_depth=7
[188]	train-auc:0.955775
[22:25:57] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 202 extra nodes, 6 pruned nodes, max_depth=7
[189]	train-auc:0.956354
[22:25:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 186 extra nodes, 6 pruned nodes, max_depth=7
[190]	train-auc:0.956876
[22:25:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 182 extra nodes, 2 pruned nodes, max_depth=7
[191]	train-auc:0.957116
[22:25:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 158 extra nodes, 0 pruned nodes, max_depth=7
[192]	train-auc:0.957507
[22:25:58] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 182 extr

...

[22:27:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 116 extra nodes, 82 pruned nodes, max_depth=7
[1169]	train-auc:0.999985
[22:27:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 104 extra nodes, 88 pruned nodes, max_depth=7
[1170]	train-auc:0.999986
[22:27:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 66 pruned nodes, max_depth=7
[1171]	train-auc:0.999986
[22:27:50] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 94 pruned nodes, max_depth=7
[1172]	train-auc:0.999986
[22:27:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 110 extra nodes, 90 pruned nodes, max_depth=7
[1173]	train-auc:0.999986
[22:27:51] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 

[22:27:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 100 extra nodes, 94 pruned nodes, max_depth=7
[1215]	train-auc:0.99999
[22:27:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 110 pruned nodes, max_depth=7
[1216]	train-auc:0.99999
[22:27:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 88 pruned nodes, max_depth=7
[1217]	train-auc:0.999989
[22:27:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 82 pruned nodes, max_depth=7
[1218]	train-auc:0.999989
[22:27:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 76 pruned nodes, max_depth=7
[1219]	train-auc:0.99999
[22:27:56] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 102 

[22:28:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 86 extra nodes, 100 pruned nodes, max_depth=7
[1261]	train-auc:0.999993
[22:28:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 92 extra nodes, 110 pruned nodes, max_depth=7
[1262]	train-auc:0.999993
[22:28:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 90 extra nodes, 82 pruned nodes, max_depth=7
[1263]	train-auc:0.999993
[22:28:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 88 pruned nodes, max_depth=7
[1264]	train-auc:0.999993
[22:28:01] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 98 extra nodes, 106 pruned nodes, max_depth=7
[1265]	train-auc:0.999993
[22:28:02] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 

[22:28:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 84 extra nodes, 102 pruned nodes, max_depth=7
[1307]	train-auc:0.999994
[22:28:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 88 extra nodes, 70 pruned nodes, max_depth=7
[1308]	train-auc:0.999994
[22:28:06] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 90 pruned nodes, max_depth=7
[1309]	train-auc:0.999994
[22:28:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 80 extra nodes, 90 pruned nodes, max_depth=7
[1310]	train-auc:0.999994
[22:28:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 94 extra nodes, 90 pruned nodes, max_depth=7
[1311]	train-auc:0.999994
[22:28:07] C:\Users\Administrator\Desktop\xgboost\src\tree\updater_prune.cc:74: tree pruning end, 1 roots, 96

XGBoostError: b'[22:28:11] C:\\Users\\Administrator\\Desktop\\xgboost\\dmlc-core\\src\\io\\local_filesys.cc:199: Check failed: allow_null  LocalFileSystem::Open "./model/xgb0.model": No such file or directory'

Output avg feature score

In [10]:
#coding=utf-8


import pandas as pd 
import os


files = os.listdir('featurescore')
fs = {}
for f in files:
    t = pd.read_csv('featurescore/'+f)
    t.index = t.feature
    t = t.drop(['feature'],axis=1)
    d = t.to_dict()['score']
    for key in d:
        if fs.__contains__(key):
            fs[key] += d[key]
        else:
            fs[key] = d[key] 
            
fs = sorted(fs.items(), key=lambda x:x[1],reverse=True)

t = []
for (key,value) in fs:
    t.append("{0},{1}\n".format(key,value))

with open('rank_feature_score.csv','w') as f:
    f.writelines("feature,score\n")
    f.writelines(t)





ouput avg score of test samples based on 10 XGBoost learner.

In [13]:
#coding=utf-8




files = os.listdir('./preds')
pred = pd.read_csv('./preds/'+files[0])
uid = pred.uid
score = pred.score
for f in files[1:]:
    pred = pd.read_csv('./preds/'+f)
    score += pred.score

score /= len(files)

pred = pd.DataFrame(uid,columns=['uid'])
pred['score'] = score
pred.to_csv('avg_preds.csv',index=None,encoding='utf-8')